<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/Using_pre_trained_CNNs_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";  # The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="5";  # Do other imports now...

from keras.models import Model
import keras.utils as image
import keras.applications.vgg16 as vgg16
import keras.applications.vgg19 as vgg19
import keras.applications.xception as xception
import keras.applications.inception_resnet_v2 as inception_resnet
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet
import keras.applications.nasnet as nasnet
import tensorflow as tf

import numpy as np
import glob
import pandas as pd

In [ ]:
def extract_feature_one_image(img_path,intermediate_layer_model,
                              model_name,input_img):

    img = image.load_img(img_path, target_size=(input_img, input_img))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)

    if(model_name == 'vgg16'):
        img_data = vgg16.preprocess_input(img_data)
    elif(model_name == 'vgg19'):
        img_data = vgg19.preprocess_input(img_data)
    elif(model_name == 'xception'):
        img_data = xception.preprocess_input(img_data)
    elif(model_name == 'resnet'):
        img_data = resnet.preprocess_input(img_data)
    elif(model_name == 'inception_resnet'):
        img_data = inception_resnet.preprocess_input(img_data)
    elif(model_name == 'nasnet'):
        img_data = nasnet.preprocess_input(img_data)
    elif(model_name == 'mobilenetv2'):
        img_data = tf.keras.applications.mobilenet_v2.preprocess_input(img_data)

    features = intermediate_layer_model.predict(img_data)
    features = features.reshape((-1))
    return features

def create_model(model_name):

    if(model_name == 'vgg16'):
        model = vgg16.VGG16(weights='imagenet', include_top=True)
        layer_name = 'fc2'
        input_image = 224
    elif(model_name == 'vgg19'):
        model = vgg19.VGG19(weights='imagenet', include_top=True)
        layer_name = 'fc2'
        input_image = 224
    elif(model_name == 'xception'):
        model = xception.Xception(weights='imagenet', include_top=True)
        layer_name = 'avg_pool'
        input_image = 299
    elif(model_name == 'resnet'):
        model = resnet.ResNet50(weights='imagenet', include_top=True)
        layer_name = 'avg_pool'
        input_image = 224
    elif(model_name == 'inception_resnet'):
        model = inception_resnet.InceptionResNetV2(weights='imagenet', include_top=True)
        layer_name = 'avg_pool'
        input_image = 299
    elif(model_name == 'nasnet'):
        model = nasnet.NASNetLarge(weights='imagenet', include_top=True)
        layer_name = 'global_average_pooling2d_1'
        input_image = 331
    elif(model_name == 'mobilenetv2'):
        model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)
        layer_name = 'global_average_pooling2d'
        input_image = 224

    #model.summary()
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)

    return intermediate_layer_model, input_image

In [ ]:
model_name = 'mobilenetv2'
#database_name = 'TOTAL_MNIST'
#database_name = 'Total_Arabic_Handwritten_Characters'
#database_name = 'shapeCN_padronizada'
#database_name = 'plantscan_padronizada'
#database_name = 'pills_balanceada_padronizada'
#database_name = 'Hands'
#database_name = 'ASL_Hand_Gestures_padronizada'
#database_name = 'swedish_padronizada'
#database_name = 'sigma_1'
#database_name = 'sigma_2'
#database_name = 'sigma_3'
#database_name = 'sigma_4'
#database_name = 'sigma_05'
database_name = 'Test'

path_database = '/content/drive/MyDrive/COVID-19/COVIDx CT CNP/' + database_name + '/'
path_write = '/content/drive/MyDrive/COVID-19/CT/Pre-trained/' + database_name + '_CNP_' + model_name + '.csv'

intermediate_layer_model, input_img = create_model(model_name)
features = []

print('Model created...')

classes = glob.glob(path_database + '*')
cont_classe = 0
features_labels_two_classes = pd.DataFrame()

for classe in classes:
    features = []
    images_names = glob.glob(classe + '/*.*')
    for name in images_names:
        features.append(extract_feature_one_image(name, intermediate_layer_model,
                                model_name,input_img))

    features_labels = pd.DataFrame(data=features)
    features_labels['Labels'] = np.uint8(cont_classe)
    features_labels_two_classes = features_labels_two_classes.append(features_labels,
                                                                 ignore_index=True)
    cont_classe+=1
features_labels_two_classes.to_csv(path_write,index=False)

print('Finished!!!!')

# Train random forest

In [ ]:
features_labels_two_classes.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_labels_two_classes.iloc[:,0:-1],
                                                    features_labels_two_classes.iloc[:,-1],
                                                    test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(database_name)
print('Accuracy = ',  accuracy_score(y_test, pred))
print('Precision = ', precision_score(y_test, pred))
print('Recall = ',  recall_score(y_test, pred))
print('F-score = ', f1_score(y_test, pred))
#print(confusion_matrix(y_test, pred))